In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys


In [2]:
# Allow imports from parent directory 
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im/35273613#35273613
#module_path = os.path.abspath(os.path.join(os.pardir))
module_path = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\programs\\'
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path

['',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\python36.zip',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\DLLs',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\agomez\\AppData\\Local\\Continuum\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\agomez\\.ipython',
 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\programs\\']

In [3]:
import time
import datetime

import pandas as pd
def ends(df, x=5):
    return df.head(x).append(df.tail(x))
setattr(pd.DataFrame,'ends',ends)
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

import seaborn as sns

import itertools
import collections
import warnings
import IPython.display
import scipy.stats
import networkx as nx
from operator import itemgetter

plt.style.use('seaborn-white')
plt.rc('font', family='serif', serif='Helvetica')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=16, linewidth=0.5)


from sklearn.preprocessing import StandardScaler, RobustScaler, FunctionTransformer, PolynomialFeatures
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import pairwise_distances


from numpy.random import exponential, negative_binomial, randint, choice, binomial
from random import shuffle

import statsmodels.formula.api as smf

LETTERS = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

import time
from scipy import interpolate

import EComm_0001_complexities 


In [4]:

# Paths
path_fig = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\figures\\'
path_data = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\data\\'
path_outputdata = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\outputdata\\'
path_inputdata = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\inputdata\\'

# format of figures
figformat = "pdf"
save2file = False

In [7]:
hdf = pd.HDFStore("https://intl-atlas-downloads.s3.amazonaws.com/data.h5")

OSError: ``https://intl-atlas-downloads.s3.amazonaws.com`` does not exist

# Loading Data

In [ ]:
classtype = "sitc"

timestring = "year"
rowvarstring = 'location_code'
rowvarstring_name = 'location_name_short_en'
colvarstring = '{classification}_product_code'.format(classification=classtype)
valtotalstring = 'export_value'
valvarstring = 'export_rca'


In [ ]:
#cpy_filepath = "https://intl-atlas-downloads.s3.amazonaws.com/country_hsproduct4digit_year.csv.zip"
classtype = "sitc"
cpy_filepath = "https://intl-atlas-downloads.s3.amazonaws.com/country_{classification}product4digit_year.csv.zip".format(classification=classtype)
ctyregions_filepath = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"

###### LOADING THE DATA #######
ctyregs_df = pd.read_csv(ctyregions_filepath)
ctyregs_df = ctyregs_df[~ctyregs_df['sub-region'].isnull()]
print(ctyregs_df.shape)

 

In [ ]:
dict_cty2reg = ctyregs_df[['region', 'sub-region']]
dict_cty2reg.index = ctyregs_df['alpha-3']
dict_cty2reg.loc[['AUT', 'USA', 'COL']].region

In [ ]:
longdf = longdf.loc[(~(longdf["pci"].isna()))]
longdf[colvarstring] = longdf[colvarstring].apply(lambda x: str(x).zfill(4))
#longdf[valtotalstring] = longdf[colvarstring].apply(lambda x: str(x).zfill(4))

In [ ]:
longdf = longdf[[timestring, rowvarstring_name, rowvarstring, colvarstring, valtotalstring, valvarstring]]

In [ ]:
# adding regional data
longdf = pd.merge(longdf, 
                 ctyregs_df[['alpha-3', 'region', 'sub-region']],
                 left_on = rowvarstring,
                 right_on = 'alpha-3',
                 how = 'left')


In [ ]:
print(longdf.shape)

## Getting the codes and checking consistency

In [ ]:
exp_codes = np.sort(longdf[rowvarstring].unique())
prod_codes = np.sort(longdf[colvarstring].unique())
iso_codes = np.sort(ctyregs_df['alpha-3'].unique())

cty_codes = np.array(list(set(exp_codes) & set(iso_codes)))
print(len(cty_codes))

longdf = longdf.loc[longdf[rowvarstring].isin(cty_codes)]
print(longdf.shape)
orig_numrows = longdf.shape[0]

In [ ]:
longdf.head()

# Dropping countries and products

In [ ]:
colvals = longdf.groupby(by = ["year", colvarstring])[valtotalstring].sum().reset_index()
colvals = colvals.drop(columns = ["year"])
print(min(colvals.iloc[:,1]))
colquants = colvals.quantile([0.01, .25, .5, .75, 0.99])
print(colquants)
colq = colquants.iloc[0][0]

In [ ]:
products2drop = np.unique(colvals.query("export_value < @colq")[colvarstring])
products2drop.shape

In [ ]:
rowvals = longdf.groupby(by = ["year", rowvarstring])[valtotalstring].sum().reset_index()
rowvals = rowvals.drop(columns = ["year"])
rowquants = rowvals.quantile([0.01, .25, .5, .75])
print(rowquants)
rowq = rowquants.iloc[0][0]

In [ ]:
countries2drop = np.unique(rowvals.query("export_value < @colq")[rowvarstring])
countries2drop.shape

### new long matrix

In [ ]:
longdf_new = longdf.loc[(~(longdf[rowvarstring].isin(countries2drop) | longdf[colvarstring].isin(products2drop)))]

In [ ]:
print(longdf_new.shape)
longdf_new.head()

# Keeping only those countries and products present in all years

In [ ]:
# products that are in all years
pcodes2keep = longdf_new.loc[((longdf_new['year']==2015) & (longdf_new[valtotalstring]>0.0))][colvarstring].unique()

# countries that are in all years
ccodes2keep = longdf_new.loc[((longdf_new['year']==2015) & (longdf_new[valtotalstring]>0.0))][rowvarstring].unique()

for yr in longdf_new.year.unique():
    pcodesyr = longdf_new.loc[((longdf_new['year']==yr) & (longdf_new[valtotalstring]>0.0))][colvarstring].unique()
    pcodes2keep = np.array(list(set(pcodes2keep) & set(pcodesyr)))
    
    ccodesyr = longdf_new.loc[((longdf_new['year']==yr) & (longdf_new[valtotalstring]>0.0))][rowvarstring].unique()
    ccodes2keep = np.array(list(set(ccodes2keep) & set(ccodesyr)))

In [ ]:
longdf_new = longdf_new[longdf_new[rowvarstring].isin(ccodes2keep) & 
                      longdf_new[colvarstring].isin(pcodes2keep)]
print((len(longdf_new.year.unique()), len(longdf_new[rowvarstring].unique()), len(longdf_new[colvarstring].unique())))

In [ ]:
# Creating totals
rowtotalvar = valtotalstring + "_{}_tot".format(rowvarstring)
coltotalvar = valtotalstring + "_{}_tot".format(colvarstring)
totaltotalvar = valtotalstring + "_year_tot"

# ---------------------
longdf_new[rowtotalvar] = longdf_new.groupby(by = ["year", rowvarstring])[valtotalstring].transform('sum')
longdf_new[coltotalvar] = longdf_new.groupby(by = ["year", colvarstring])[valtotalstring].transform('sum')
longdf_new[totaltotalvar] = longdf_new.groupby(by = ["year"])[valtotalstring].transform('sum')

longdf_new = longdf_new.loc[(~( (longdf_new[rowtotalvar]<=1000.0) | (longdf_new[coltotalvar]<=1000.0) ))]



In [ ]:
longdf_new = longdf_new.drop(columns=[rowtotalvar, coltotalvar, totaltotalvar])
# ---------------------
longdf_new.loc[:,rowtotalvar] = pd.Series(longdf_new.groupby(by = ["year", rowvarstring])[valtotalstring].transform('sum'))
longdf_new.loc[:,coltotalvar] = pd.Series(longdf_new.groupby(by = ["year", colvarstring])[valtotalstring].transform('sum'))
longdf_new.loc[:,totaltotalvar] = pd.Series(longdf_new.groupby(by = ["year"])[valtotalstring].transform('sum'))


In [ ]:
final_numrows = longdf_new.shape[0]

print("The number of rows erased was {}.".format(orig_numrows - final_numrows))
longdf_new.shape

In [ ]:
(len(longdf_new["year"].unique()), len(longdf_new[rowvarstring].unique()), len(longdf_new[colvarstring].unique()))

In [ ]:
longdf_new.head()

In [ ]:
longdf_new.groupby('year', as_index=False).agg({rowvarstring: 'nunique', colvarstring: 'nunique'}).ends(5)

## Preparation

In [ ]:
# pivon and un-pivot
newdf = longdf_new.pivot_table(index=["year", rowvarstring],
                            columns=colvarstring,
                            values=valtotalstring).fillna(value=0)

In [ ]:
longdf_new = pd.merge(newdf.reset_index(level=("year", rowvarstring)).melt(id_vars=["year", rowvarstring], 
            value_vars=newdf.columns.tolist(),
            value_name=valtotalstring)[["year", rowvarstring,colvarstring]],
                   longdf_new,
                 how = 'left')
longdf_new.ends(2)

In [ ]:
%reset Out

In [ ]:
# Creating totals
rowtotalvar = valtotalstring + "_{}_tot".format(rowvarstring)
coltotalvar = valtotalstring + "_{}_tot".format(colvarstring)
totaltotalvar = valtotalstring + "_year_tot"

# ---------------------
longdf_new[rowtotalvar] = longdf_new.groupby(by = ["year", rowvarstring])[valtotalstring].transform('sum')
longdf_new[coltotalvar] = longdf_new.groupby(by = ["year", colvarstring])[valtotalstring].transform('sum')
longdf_new[totaltotalvar] = longdf_new.groupby(by = ["year"])[valtotalstring].transform('sum')

longdf_new = longdf_new.loc[(~( (longdf_new[rowtotalvar]<=1000.0) | (longdf_new[coltotalvar]<=1000.0) ))]



In [ ]:
# Creating logarithms
longdf_new["constant"] = 1
longdf_new["log_" + valtotalstring] = np.log(1 + longdf_new[valtotalstring].values)
longdf_new[["log_" + rowtotalvar, "log_" + coltotalvar, "log_" + totaltotalvar]] = np.log(longdf_new[[rowtotalvar, coltotalvar, totaltotalvar]])


In [ ]:
Xmat = longdf_new[["log_" + rowtotalvar, "log_" + coltotalvar, "log_" + totaltotalvar]].values
ymat = longdf_new["log_" + valtotalstring].values
reg = LinearRegression().fit(Xmat, ymat)

In [ ]:
# creating the residuals of a regression
rcaval = "rca_ols"
mcpval = "mcp_ols"
mcporigval = "mcp_orig"
longdf_new[rcaval] = ymat - reg.predict(Xmat)  
longdf_new[mcpval] = (longdf_new[rcaval] > 0).astype(int)  
longdf_new[mcporigval] = (longdf_new[valvarstring] > 1).astype(int)  


In [ ]:
longdf_new.loc[(longdf_new["year"]==2010)].loc[longdf_new[valvarstring]>0][[valvarstring,rcaval]].describe()

In [ ]:
sns.distplot(longdf_new.loc[(longdf_new["year"]==2010)].loc[longdf_new[valvarstring]>0][rcaval].values, kde=True, rug=True);

In [ ]:
x = longdf_new.loc[(longdf_new["year"]==2010)][rcaval].values
y = longdf_new.loc[(longdf_new["year"]==2010)][valvarstring].values

In [ ]:
plt.semilogy(x, y, marker = '.', linestyle="", markersize=0.5)
plt.show()

In [ ]:
longdf_new.shape

# Wide matrix version

In [ ]:
widedf_rca = longdf_new.pivot_table(index=["year", rowvarstring],
                            columns=colvarstring,
                            values=mcpval).fillna(value=0)


# Complexity Spaces

In [ ]:
yr = 2015
# subsetting to year
widemat_yr = widedf_rca.query("year == @yr")
rsum = widemat_yr.sum(axis=1)
csum = widemat_yr.sum(axis=0)
irsum = np.argsort(-rsum)
icsum = np.argsort(-csum)
ctrylist_inorder = np.array(list(zip(*widemat_yr.index.values))[1])[irsum]
prodlist_inorder = widemat_yr.columns.values[icsum]

# reordering
widemat_yr = widemat_yr.reset_index(level=["year"], drop=True).loc[ctrylist_inorder, prodlist_inorder]


In [ ]:
(Mc2c, Dc, leftVc, rightVc), (Mp2p, Dp, leftVp, rightVp) = EComm_0001_complexities.ECeigenvecs(widemat_yr)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0.2)

ax1 = fig.add_subplot(2,2,1)
# histogram of country eigenvalues
sns.distplot(Dc.real, bins=50, rug=True, kde=False, norm_hist=True, ax=ax1, axlabel="Eigenvalues")
sns.kdeplot(Dc.real, bw=.005, ax=ax1)
ax1.set_ylabel("Statistical Frequency")

ax2 = fig.add_subplot(2,2,2)
# histogram of country ECI's
sns.distplot(leftVc[:,1].real, bins=40, rug=True, kde=False, norm_hist=True, ax=ax2, axlabel="2nd left-eigenvector (aka, ECI)")
sns.kdeplot(leftVc[:,1].real, bw=.01, ax=ax2)

ax3 = fig.add_subplot(2,2,3)
# histogram of country eigenvalues
sns.distplot(Dp.real, bins=50, rug=True, kde=False, norm_hist=True, ax=ax3, axlabel="Eigenvalues")
sns.kdeplot(Dp.real, bw=.005, ax=ax3)
ax3.set_ylabel("Statistical Frequency")

ax4 = fig.add_subplot(2,2,4)
# histogram of country ECI's
sns.distplot(leftVp[:,1].real, bins=40, rug=True, kde=False, norm_hist=True, ax=ax4, axlabel="2nd left-eigenvector (aka, PCI)")
sns.kdeplot(leftVp[:,1].real, bw=.01, ax=ax4)


plt.show()


# UMAP

In [ ]:
import umap

# Creating a figure

In [ ]:
%reset Out

In [ ]:
def plotUMAP(widedf_rca, yr, ctrylist_inorder, prodlist_inorder, umap_reducer, dict_cty2reg,
             change=False, pathname_fig="myfig.png", save=False, **kwargs):

    #yr = 2015

    # subsetting to year
    widemat_yr = widedf_rca.query("year == @yr")

    # reordering
    widemat_yr = widemat_yr.reset_index(level=["year"], drop=True).loc[ctrylist_inorder, prodlist_inorder]

    # applying umap
    embedding_yr = umap_reducer.transform(widemat_yr)

    plttitle = "Matrix of presences at time {t}".format(t=yr)
    if change==True:
        # subsetting to previous year
        widemat_prevyr = widedf_rca.query("year == @yr - 1")
        widemat_prevyr = widemat_prevyr.reset_index(level=["year"], drop=True).loc[ctrylist_inorder, prodlist_inorder]
        # The change in the matrix
        widemat_yr = widemat_yr - widemat_prevyr
        plttitle = "Matrix change from year {tprev} to {t}".format(tprev = yr-1, t=yr)

    # Creating plot
    fig = plt.figure(figsize=(24,7))

    ax1 = fig.add_subplot(121)
    im = ax1.imshow(-widemat_yr, aspect=4, cmap = 'bwr')
    ax1.set_title(plttitle, fontsize=25)
    fig.colorbar(im, ax=ax1, ticks=np.linspace(-1,1,1))
    #ax1 = plt.tight_layout()

    ax2 = fig.add_subplot(122)
    ax2.scatter(embedding_yr[:, 0], embedding_yr[:, 1], 
                marker = ".",
                **kwargs)
    for i, row in enumerate(embedding_yr):
        plt.text(row[0], row[1], ctrylist_inorder[i], fontsize = 10, bbox={'facecolor': "black", 'alpha':0.3, 'pad':3})
    plt.gca().set_aspect('equal', 'datalim')
    plt.title('UMAP projection', fontsize=24)

    if save:
        fig.savefig(pathname_fig)
        plt.close()
    else:
        plt.show()

## Fitting UMAP

In [ ]:
# Setting initial year
yr = 2015

# subsetting to year
widemat_yr = widedf_rca.query("year == @yr")
rsum = widemat_yr.sum(axis=1)
csum = widemat_yr.sum(axis=0)
irsum = np.argsort(-rsum)
icsum = np.argsort(-csum)
ctrylist_inorder = np.array(list(zip(*widemat_yr.index.values))[1])[irsum]
prodlist_inorder = widemat_yr.columns.values[icsum]

# reordering
widemat_yr = widemat_yr.reset_index(level=["year"], drop=True).loc[ctrylist_inorder, prodlist_inorder]

# fitting umap
#umap_reducer = umap.UMAP(random_state=42,
#                         metric='jaccard',
#                         n_neighbors = 30,
#                         min_dist = 0.25).fit(widemat_yr)


In [ ]:
cmapvir = plt.get_cmap('viridis')
colors = cmapvir(np.linspace(0,1,len(dict_cty2reg.loc[ctrylist_inorder].region)))
clr = [colors[i] for i in range(len(dict_cty2reg.loc[ctrylist_inorder].region))]

In [ ]:
# fitting umap
yr = 2015
umap_reducer = umap.UMAP(random_state=42, n_epochs=1000,
                         metric='jaccard',
                         n_neighbors = 10,
                         min_dist = 1).fit(widemat_yr)
plotUMAP(widedf_rca, yr, ctrylist_inorder, prodlist_inorder, umap_reducer, dict_cty2reg,
         change=True, c = clr)

In [ ]:
yr = 2015
plotUMAP(widedf_rca, yr, ctrylist_inorder, prodlist_inorder, umap_reducer)

In [ ]:
allyears = np.sort(longdf_new.year.unique())
if False:
    for yr in allyears:
        plotUMAP(widedf_rca, yr, ctrylist_inorder, prodlist_inorder, umap_reducer, 
             pathname_fig = path_fig + "test{t}".format(t=yr) + ".png", save=True)

# creating movie

In [ ]:
import imageio

allyears = np.sort(longdf_new.year.unique())
filenames = [path_fig + "test{t}".format(t=yr) + ".png" for yr in allyears]

with imageio.get_writer(path_fig + "DM_0002_movie.gif", mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
from sklearn.datasets import load_digits
import tempfile
test_data_home = tempfile.mkdtemp()

In [ ]:
test_data_home

In [ ]:
mnist = load_digits()


In [ ]:
mnist.target

In [ ]:
mnist.data.shape

In [ ]:
sns.set(context="paper", style="white")


reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(mnist.data)



In [ ]:
embedding

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
plt.scatter(
    embedding[:, 0], embedding[:, 1], 
    c=mnist.target, 
    cmap="Spectral", 
    #s=0.1, 
    marker='o',
)
#plt.setp(ax, xticks=[], yticks=[])
plt.title("MNIST data embedded into two dimensions by UMAP", fontsize=18)

plt.show()

In [ ]:
int(dict_cty2reg.loc[ctrylist_inorder].region)

In [ ]:
clr